# Default notebook

This default notebook is executed using a Lakeflow job as defined in resources/sample_job.job.yml.

In [0]:
from var_etl.pipeline.silver_market_returns import compute_returns, upsert_returns
from datetime import datetime, date

catalog = dbutils.widgets.get("catalog")
schema  = dbutils.widgets.get("schema")
target_table = dbutils.widgets.get("target_table")
silver_table = dbutils.widgets.get("silver_table")
as_of_date_raw = dbutils.widgets.get("as_of_date")

if not as_of_date_raw or as_of_date_raw.lower() == "none":
    as_of_date = date.today()
else:
    as_of_date = datetime.strptime(as_of_date_raw[:10], "%Y-%m-%d").date()


spark.sql(f"USE CATALOG `{catalog}`")
spark.sql(f"USE SCHEMA `{schema}`")
df = spark.read.format("delta").table(target_table)
df_returns = compute_returns(
    df=df
)
max_available_date = upsert_returns(df_return=df_returns, spark=spark, silver_table=silver_table)
should_run_gold = as_of_date <= max_available_date
dbutils.jobs.taskValues.set(
    key="should_run_gold",
    value=str(should_run_gold).lower()
)
print(f"should_run_gold={should_run_gold} (as_of_date={as_of_date} <= max_available_date={max_available_date})")
if not should_run_gold:
    dbutils.notebook.exit(
        f"SKIP: as_of_date={as_of_date_raw} > max_available_date={max_available_date}"
    )
